## Capstone Segmenation and Clustering Dataframe

### Latitude and Longtitude coordinate data added to the postal codes

Run through generating the dataframe of Canadian PostalCodes, Boroughs and Neighborhoods from the Capstone Segmenation and Clustering Dataframe example

In [212]:
import pandas as pd
!pip install lxml
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
postalcode_table = dfs[0]
column_names = ['PostalCode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)
for indices, columns in postalcode_table.iterrows():
    for CellValue in columns:
        PostalCode = CellValue[0:3]
        Neighborhood_Start = ''                         
        if CellValue.find("(") == -1:                   
            Neighborhood_Start = len(CellValue)
        else:
            Neighborhood_Start = CellValue.find("(")    
        Borough = CellValue[3:Neighborhood_Start]
        Neighborhood = (CellValue[Neighborhood_Start+1:].replace(" /",",")).replace(")","")
        if Borough == 'Not assigned':
            pass                     
        else:
            neighborhoods = neighborhoods.append({'PostalCode' : PostalCode,
                                                  'Borough' : Borough,
                                                  'Neighborhood' : Neighborhood}, ignore_index=True)
neighborhoods.shape #confirm size of frame is as expected

(103, 3)

In [213]:
# install and import geocoder
!pip install geocoder
import geocoder

In [214]:
# set up an empty dataframe to hold our results
# define the required columns
column_names = ['PostalCode','Latitude','Longtitude']

# create the results dataframe
Geospatial_Coordinates = pd.DataFrame(columns=column_names)
Geospatial_Coordinates

,PostalCode,Latitude,Longtitude


Unfortunately, using the .google source as instructed in the course material does not return results,
so the osm source has been used.  This gives slightly different results from those given in the csv file
although reasonably close.

In [215]:
# loop through the locations in the neighborhoods data

for indices,postalcode in neighborhoods.iterrows():
    lat_lng_coords = None
    latitude = None
    longtitude = None
    location = postalcode[2].split(",")[0] + ", " + 'Canada'
    g = geocoder.osm(location)
    lat_lng_coords = g.latlng
    if lat_lng_coords != None:
        latitude = lat_lng_coords[0]
        longtitude = lat_lng_coords[1]
    Geospatial_Coordinates = Geospatial_Coordinates.append({'PostalCode' : postalcode[0],
                                                            'Latitude' : latitude,
                                                            'Longtitude' : longtitude}, ignore_index=True)
Geospatial_Coordinates.shape #confirm size of frame is as expected

(103, 3)

Now we will import the csv file so that the results are consistent with those expected

In [216]:
#!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
supplied_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
supplied_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will now combine the data from the downloaded file with the neighborhoods dataframe

In [217]:
combined_columns = ['PostalCode','Borough','Neighborhood','Latitude','Longtitude']
Canadian_Geospatial_Coordinates = pd.DataFrame(columns=combined_columns)
Canadian_Geospatial_Coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longtitude


In [219]:
# Update the supplied_cordinates column name ahead of the merge
supplied_coordinates=supplied_coordinates.rename(columns={'Postal Code': 'PostalCode'})

In [220]:
Canadian_Geospatial_Coordinates = pd.merge(neighborhoods,supplied_coordinates[['PostalCode','Latitude','Longitude']],on='PostalCode',how='left')
Canadian_Geospatial_Coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
